### 0. Import Libs

In [1]:
# kernel must be msd2
import constant as c
import os
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
import warnings; warnings.filterwarnings('ignore')
import pandas as pd


### 1. Make Grids

In [2]:
# Load the shapefile of Seoul
seoul_shapefile = os.path.join(c.SSD_PATH, 'nsdi/LARD_ADM_SECT_SGG_서울/LARD_ADM_SECT_SGG_11.shp')
seoul = gpd.read_file(seoul_shapefile)

# Get the bounding box of Seoul
minx, miny, maxx, maxy = seoul.total_bounds

# Set the grid size (100m x 100m)
grid_size = 100

# Calculate the number of rows and columns
rows = int((maxy - miny) / grid_size)
cols = int((maxx - minx) / grid_size)

# Create the grid
grid = []
for i in range(cols):
    for j in range(rows):
        square = Polygon([
            (minx + i * grid_size, miny + j * grid_size),
            (minx + (i + 1) * grid_size, miny + j * grid_size),
            (minx + (i + 1) * grid_size, miny + (j + 1) * grid_size),
            (minx + i * grid_size, miny + (j + 1) * grid_size),
        ])
        grid.append(square)

# Convert the grid to a GeoDataFrame
grid_gdf = gpd.GeoDataFrame({"geometry": grid})

# Set the coordinate reference system (CRS) of the grid to match Seoul's CRS
grid_gdf.crs = seoul.crs

# Clip the grid to the shape of Seoul
grid_clipped = gpd.overlay(grid_gdf, seoul, how="intersection")

# Extract only polygon object with 'square'
def is_square(polygon):
    if isinstance(polygon, Polygon) and len(polygon.exterior.coords) == 5:
        coords = list(polygon.exterior.coords)[:-1]
        edges = [(coords[i][0] - coords[i - 1][0], coords[i][1] - coords[i - 1][1]) for i in range(4)]
        edge_lengths = [abs(complex(*e)) for e in edges]
        return all(abs(edge_lengths[i] - edge_lengths[i - 1]) < 1e-6 for i in range(4)) and (abs(edges[0][0] * edges[1][0] + edges[0][1] * edges[1][1]) < 1e-6)
    return False

def extract_squares(geometry):
    if isinstance(geometry, Polygon):
        if is_square(geometry):
            return geometry
    elif isinstance(geometry, MultiPolygon):
        squares = [p for p in geometry if is_square(p)]
        if squares:
            return MultiPolygon(squares)
    return None

gdf_squares = grid_clipped.copy()
gdf_squares['geometry'] = gdf_squares.geometry.apply(extract_squares)
gdf_squares.dropna(subset=['geometry'], inplace=True)

# give index
gdf_squares['idx'] = range(1, len(gdf_squares) + 1)
gdf_squares['idx'] = gdf_squares['idx'].astype(int)

# extract only gus in three bds
df_grid = gdf_squares[gdf_squares['COL_ADM_SE'].isin(c.CBD_CDS)]
df_grid = df_grid[c.GRID_DF_COL]
df_grid.to_csv('../asset/preprocess/grid/df_grid.csv')

In [3]:
df_grid.head(10)

,ADM_SECT_C,geometry,idx
12512,11560,"POLYGON ((945135.230 1946765.548, 945135.230 1...",10658
12517,11560,"POLYGON ((945135.230 1947265.548, 945135.230 1...",10659
12518,11560,"POLYGON ((945135.230 1947365.548, 945135.230 1...",10660
12523,11560,"POLYGON ((945135.230 1950365.548, 945135.230 1...",10661
12524,11560,"POLYGON ((945135.230 1950465.548, 945135.230 1...",10662
12531,11560,"POLYGON ((945235.230 1946665.548, 945235.230 1...",10663
12532,11560,"POLYGON ((945235.230 1946765.548, 945235.230 1...",10664
12533,11560,"POLYGON ((945235.230 1946865.548, 945235.230 1...",10665
12534,11560,"POLYGON ((945235.230 1946965.548, 945235.230 1...",10666
12535,11560,"POLYGON ((945235.230 1947065.548, 945235.230 1...",10667


In [4]:
# Read df_grid
df_grid = pd.read_csv('../asset/preprocess/grid/df_grid.csv', index_col = 0)
df_grid.head(10)

,ADM_SECT_C,geometry,idx
12512,11560,"POLYGON ((945135.230213883 1946765.54807064, 9...",10658
12517,11560,"POLYGON ((945135.230213883 1947265.54807064, 9...",10659
12518,11560,"POLYGON ((945135.230213883 1947365.54807064, 9...",10660
12523,11560,"POLYGON ((945135.230213883 1950365.54807064, 9...",10661
12524,11560,"POLYGON ((945135.230213883 1950465.54807064, 9...",10662
12531,11560,"POLYGON ((945235.230213883 1946665.54807064, 9...",10663
12532,11560,"POLYGON ((945235.230213883 1946765.54807064, 9...",10664
12533,11560,"POLYGON ((945235.230213883 1946865.54807064, 9...",10665
12534,11560,"POLYGON ((945235.230213883 1946965.54807064, 9...",10666
12535,11560,"POLYGON ((945235.230213883 1947065.54807064, 9...",10667
